# Extreme Gradient Boosting with XGBoost - Part 4
- Select Categorical Feature: df.select_dtypes(include=[object])
- Select Numerical Feature: df.select_dtypes(include=[np.number])
- LabelEncoder
    - Encode categorical features to numeric
    - Need manually call fit_transform method to each column
    - classes_ attribute return all unique encoded values
- OneHotEncoder
    - Encode categorical features as a one-hot numeric
    - Accept only categical features on input
    - categories_ attribute return all categories of each feature
    - get_feature_names method return encoded column names
- DictVectorizer
    - Encode categorical features as a one-hot numeric
    - Accept categical and numerical features on input
    - Require array of dictionaries on input (converted from DataFrame)
    - vocabulary_ attribute return feature names with feature indices
    - get_feature_names method return encoded column names
- ColumnTransformer
    - Encode categorical features as a one-hot numeric
    - Accept categical features column names on input
    - get_feature_names method return encoded column names
- FeatureUnion

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import xgboost as xgb

from sklearn_pandas import DataFrameMapper
from sklearn_pandas import CategoricalImputer
from sklearn.impute import SimpleImputer

from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin

## Datasets

### Boston Housing

In [2]:
from sklearn.datasets import load_boston
boston = load_boston()

### Ames Housing Prices

In [3]:
ames = pd.read_csv('../datasets/ames_housing_prices.csv')

### Chronic Kidney Disease

In [4]:
kidney_disease = pd.read_csv('../datasets/chronic_kidney_disease.csv', na_values='?')

## Using XGBoost in pipelines

### Encoding categorical columns I: LabelEncoder
Now that you've seen what will need to be done to get the housing data ready for XGBoost, let's go through the process step-by-step.

First, you will need to fill in missing values - as you saw previously, the column LotFrontage has many missing values. Then, you will need to encode any categorical columns in the dataset using one-hot encoding so that they are encoded numerically. You can watch this video from Supervised Learning with scikit-learn for a refresher on the idea.

The data has five categorical columns: MSZoning, PavedDrive, Neighborhood, BldgType, and HouseStyle. Scikit-learn has a LabelEncoder function that converts the values in each categorical column into integers. You'll practice using this here.

In [5]:
df = ames.copy()

In [6]:
# Fill missing values with 0
df.LotFrontage = df.LotFrontage.fillna(0)

In [7]:
# Create a boolean mask for categorical columns
categorical_mask = (df.dtypes == object)

In [17]:
# Get list of categorical column names
categorical_columns = df.columns[categorical_mask].tolist()

In [18]:
# Print the head of the categorical columns
print(df[categorical_columns].head())

  MSZoning Neighborhood BldgType HouseStyle PavedDrive
0       RL      CollgCr     1Fam     2Story          Y
1       RL      Veenker     1Fam     1Story          Y
2       RL      CollgCr     1Fam     2Story          Y
3       RL      Crawfor     1Fam     2Story          Y
4       RL      NoRidge     1Fam     2Story          Y


In [19]:
# Create LabelEncoder object: le
le = LabelEncoder()

In [20]:
# Apply LabelEncoder to categorical columns
df[categorical_columns] = df[categorical_columns].apply(lambda x: le.fit_transform(x))

In [21]:
# Print the head of the LabelEncoded categorical columns
print(df[categorical_columns].head())

   MSZoning  Neighborhood  BldgType  HouseStyle  PavedDrive
0         3             5         0           5           2
1         3            24         0           2           2
2         3             5         0           5           2
3         3             6         0           5           2
4         3            15         0           5           2


### Encoding categorical columns II: OneHotEncoder
Okay - so you have your categorical columns encoded numerically. Can you now move onto using pipelines and XGBoost? Not yet! In the categorical columns of this dataset, there is no natural ordering between the entries. As an example: Using LabelEncoder, the CollgCr Neighborhood was encoded as 5, while the Veenker Neighborhood was encoded as 24, and Crawfor as 6. Is Veenker "greater" than Crawfor and CollgCr? No - and allowing the model to assume this natural ordering may result in poor performance.

As a result, there is another step needed: You have to apply a one-hot encoding to create binary, or "dummy" variables. You can do this using scikit-learn's OneHotEncoder.

In [22]:
df = ames.copy()

In [118]:
# Create OneHotEncoder: ohe
ohe = OneHotEncoder(categorical_features=categorical_mask, sparse=False)

# categorical_features: Deprecated since version 0.20: 
# The categorical_features keyword was deprecated in version 0.20 and will be removed in 0.22. 
# You can use the ColumnTransformer instead.

TypeError: __init__() got an unexpected keyword argument 'categorical_features'

In [23]:
# Create OneHotEncoder: ohe
ohe = OneHotEncoder(sparse=False)

In [24]:
# Apply OneHotEncoder to categorical columns - output is no longer a dataframe: df_encoded
df_encoded = ohe.fit_transform(df[categorical_columns])

In [27]:
# Get list of non-categorical column names
non_categorical_columns = df.columns[~categorical_mask].tolist()

In [28]:
# Concatenate categorical and non-categorical columns
df_encoded = np.concatenate([df_encoded, df[non_categorical_columns].values], axis=1)

In [29]:
# Print first 5 rows of the resulting dataset - again, this will no longer be a pandas dataframe
print(df_encoded[:5, :])

[[0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 1.000e+00 6.000e+01 6.500e+01 8.450e+03
  7.000e+00 5.000e+00 2.003e+03 0.000e+00 1.710e+03 1.000e+00 0.000e+00
  2.000e+00 1.000e+00 3.000e+00 0.000e+00 5.480e+02 2.085e+05]
 [0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 1.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 

In [30]:
# Print the shape of the original DataFrame
print(df.shape)

(1460, 21)


In [31]:
# Print the shape of the transformed array
print(df_encoded.shape)

(1460, 62)


In [32]:
# Categories from transformed data
ohe.categories_

[array(['C (all)', 'FV', 'RH', 'RL', 'RM'], dtype=object),
 array(['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr',
        'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel',
        'NAmes', 'NPkVill', 'NWAmes', 'NoRidge', 'NridgHt', 'OldTown',
        'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber',
        'Veenker'], dtype=object),
 array(['1Fam', '2fmCon', 'Duplex', 'Twnhs', 'TwnhsE'], dtype=object),
 array(['1.5Fin', '1.5Unf', '1Story', '2.5Fin', '2.5Unf', '2Story',
        'SFoyer', 'SLvl'], dtype=object),
 array(['N', 'P', 'Y'], dtype=object)]

In [35]:
# Column names after encoding
ohe.get_feature_names(df[categorical_columns].columns.values)

array(['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL',
       'MSZoning_RM', 'Neighborhood_Blmngtn', 'Neighborhood_Blueste',
       'Neighborhood_BrDale', 'Neighborhood_BrkSide',
       'Neighborhood_ClearCr', 'Neighborhood_CollgCr',
       'Neighborhood_Crawfor', 'Neighborhood_Edwards',
       'Neighborhood_Gilbert', 'Neighborhood_IDOTRR',
       'Neighborhood_MeadowV', 'Neighborhood_Mitchel',
       'Neighborhood_NAmes', 'Neighborhood_NPkVill',
       'Neighborhood_NWAmes', 'Neighborhood_NoRidge',
       'Neighborhood_NridgHt', 'Neighborhood_OldTown',
       'Neighborhood_SWISU', 'Neighborhood_Sawyer',
       'Neighborhood_SawyerW', 'Neighborhood_Somerst',
       'Neighborhood_StoneBr', 'Neighborhood_Timber',
       'Neighborhood_Veenker', 'BldgType_1Fam', 'BldgType_2fmCon',
       'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE',
       'HouseStyle_1.5Fin', 'HouseStyle_1.5Unf', 'HouseStyle_1Story',
       'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'HouseStyle_2

### Encoding categorical columns III: DictVectorizer
Alright, one final trick before you dive into pipelines. The two step process you just went through - LabelEncoder followed by OneHotEncoder - can be simplified by using a DictVectorizer.

Using a DictVectorizer on a DataFrame that has been converted to a dictionary allows you to get label encoding as well as one-hot encoding in one go.

Your task is to work through this strategy in this exercise!

In [36]:
df = ames.copy()

In [37]:
# Convert df into a dictionary: df_dict
df_dict = df.to_dict(orient='records')

In [38]:
# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse=False)

In [39]:
# Apply dv on df: df_encoded
df_encoded = dv.fit_transform(df_dict)

In [40]:
# Print the resulting first five rows
print(df_encoded[:5,:])

[[3.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 2.000e+00 5.480e+02 1.710e+03 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00
  8.450e+03 6.500e+01 6.000e+01 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 5.000e+00 7.000e+00
  0.000e+00 0.000e+00 1.000e+00 0.000e+00 2.085e+05 2.003e+03]
 [3.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.000e+00 1.000e+00 2.000e+00 4.600e+02 1.262e+03 0.000e+00 0.000e+00
  0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  9.600e+03 8.000e+01 2.000e+01 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 

In [41]:
# Print the vocabulary
print(dv.vocabulary_)

{'MSSubClass': 23, 'MSZoning=RL': 27, 'LotFrontage': 22, 'LotArea': 21, 'Neighborhood=CollgCr': 34, 'BldgType=1Fam': 1, 'HouseStyle=2Story': 18, 'OverallQual': 55, 'OverallCond': 54, 'YearBuilt': 61, 'Remodeled': 59, 'GrLivArea': 11, 'BsmtFullBath': 6, 'BsmtHalfBath': 7, 'FullBath': 9, 'HalfBath': 12, 'BedroomAbvGr': 0, 'Fireplaces': 8, 'GarageArea': 10, 'PavedDrive=Y': 58, 'SalePrice': 60, 'Neighborhood=Veenker': 53, 'HouseStyle=1Story': 15, 'Neighborhood=Crawfor': 35, 'Neighborhood=NoRidge': 44, 'Neighborhood=Mitchel': 40, 'HouseStyle=1.5Fin': 13, 'Neighborhood=Somerst': 50, 'Neighborhood=NWAmes': 43, 'MSZoning=RM': 28, 'Neighborhood=OldTown': 46, 'Neighborhood=BrkSide': 32, 'BldgType=2fmCon': 2, 'HouseStyle=1.5Unf': 14, 'Neighborhood=Sawyer': 48, 'Neighborhood=NridgHt': 45, 'Neighborhood=NAmes': 41, 'BldgType=Duplex': 3, 'Neighborhood=SawyerW': 49, 'Neighborhood=IDOTRR': 38, 'PavedDrive=N': 56, 'Neighborhood=MeadowV': 39, 'BldgType=TwnhsE': 5, 'MSZoning=C (all)': 24, 'Neighborhood=E

In [48]:
# Column names after encoding
print(dv.get_feature_names())

['BedroomAbvGr', 'BldgType=1Fam', 'BldgType=2fmCon', 'BldgType=Duplex', 'BldgType=Twnhs', 'BldgType=TwnhsE', 'BsmtFullBath', 'BsmtHalfBath', 'Fireplaces', 'FullBath', 'GarageArea', 'GrLivArea', 'HalfBath', 'HouseStyle=1.5Fin', 'HouseStyle=1.5Unf', 'HouseStyle=1Story', 'HouseStyle=2.5Fin', 'HouseStyle=2.5Unf', 'HouseStyle=2Story', 'HouseStyle=SFoyer', 'HouseStyle=SLvl', 'LotArea', 'LotFrontage', 'MSSubClass', 'MSZoning=C (all)', 'MSZoning=FV', 'MSZoning=RH', 'MSZoning=RL', 'MSZoning=RM', 'Neighborhood=Blmngtn', 'Neighborhood=Blueste', 'Neighborhood=BrDale', 'Neighborhood=BrkSide', 'Neighborhood=ClearCr', 'Neighborhood=CollgCr', 'Neighborhood=Crawfor', 'Neighborhood=Edwards', 'Neighborhood=Gilbert', 'Neighborhood=IDOTRR', 'Neighborhood=MeadowV', 'Neighborhood=Mitchel', 'Neighborhood=NAmes', 'Neighborhood=NPkVill', 'Neighborhood=NWAmes', 'Neighborhood=NoRidge', 'Neighborhood=NridgHt', 'Neighborhood=OldTown', 'Neighborhood=SWISU', 'Neighborhood=Sawyer', 'Neighborhood=SawyerW', 'Neighborhoo

### Encoding categorical columns IV: ColumnTransformer
Alternative to LabelEncoder with OneHotEncoder

In [50]:
df = ames.copy()

In [51]:
# Fill missing values with 0
df.LotFrontage = df.LotFrontage.fillna(0)

In [52]:
# Create a boolean mask for categorical columns
categorical_mask = (df.dtypes == object)

In [53]:
# Get list of categorical column names
categorical_columns = df.columns[categorical_mask].tolist()

In [59]:
# Create the ColumnTransformer object
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), categorical_columns)])

In [54]:
# Create the ColumnTransformer object
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), categorical_columns)], remainder='passthrough')

In [60]:
# Apply OneHotEncoder to categorical columns 
df_encoded = columnTransformer.fit_transform(df)

In [61]:
# Print the resulting first five rows
print(df_encoded[:5,:])

  (0, 3)	1.0
  (0, 10)	1.0
  (0, 30)	1.0
  (0, 40)	1.0
  (0, 45)	1.0
  (1, 3)	1.0
  (1, 29)	1.0
  (1, 30)	1.0
  (1, 37)	1.0
  (1, 45)	1.0
  (2, 3)	1.0
  (2, 10)	1.0
  (2, 30)	1.0
  (2, 40)	1.0
  (2, 45)	1.0
  (3, 3)	1.0
  (3, 11)	1.0
  (3, 30)	1.0
  (3, 40)	1.0
  (3, 45)	1.0
  (4, 3)	1.0
  (4, 20)	1.0
  (4, 30)	1.0
  (4, 40)	1.0
  (4, 45)	1.0


In [62]:
# Print the shape of the transformed array
print(df_encoded.shape)

(1460, 46)


In [64]:
# Column names after encoding
print(columnTransformer.get_feature_names())

['encoder__x0_C (all)', 'encoder__x0_FV', 'encoder__x0_RH', 'encoder__x0_RL', 'encoder__x0_RM', 'encoder__x1_Blmngtn', 'encoder__x1_Blueste', 'encoder__x1_BrDale', 'encoder__x1_BrkSide', 'encoder__x1_ClearCr', 'encoder__x1_CollgCr', 'encoder__x1_Crawfor', 'encoder__x1_Edwards', 'encoder__x1_Gilbert', 'encoder__x1_IDOTRR', 'encoder__x1_MeadowV', 'encoder__x1_Mitchel', 'encoder__x1_NAmes', 'encoder__x1_NPkVill', 'encoder__x1_NWAmes', 'encoder__x1_NoRidge', 'encoder__x1_NridgHt', 'encoder__x1_OldTown', 'encoder__x1_SWISU', 'encoder__x1_Sawyer', 'encoder__x1_SawyerW', 'encoder__x1_Somerst', 'encoder__x1_StoneBr', 'encoder__x1_Timber', 'encoder__x1_Veenker', 'encoder__x2_1Fam', 'encoder__x2_2fmCon', 'encoder__x2_Duplex', 'encoder__x2_Twnhs', 'encoder__x2_TwnhsE', 'encoder__x3_1.5Fin', 'encoder__x3_1.5Unf', 'encoder__x3_1Story', 'encoder__x3_2.5Fin', 'encoder__x3_2.5Unf', 'encoder__x3_2Story', 'encoder__x3_SFoyer', 'encoder__x3_SLvl', 'encoder__x4_N', 'encoder__x4_P', 'encoder__x4_Y']


### Preprocessing within a pipeline
Now that you've seen what steps need to be taken individually to properly process the Ames housing data, let's use the much cleaner and more succinct DictVectorizer approach and put it alongside an XGBoostRegressor inside of a scikit-learn pipeline.

In [21]:
df = ames.copy()
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [31]:
# Fill LotFrontage missing values with 0
X.LotFrontage = X.LotFrontage.fillna(0)

In [32]:
# Setup the pipeline steps: steps
steps = [("ohe_onestep", DictVectorizer(sparse=False)),
         ("xgb_model", xgb.XGBRegressor())]

In [33]:
# Create the pipeline: xgb_pipeline
xgb_pipeline = Pipeline(steps)

In [34]:
# Fit the pipeline
xgb_pipeline.fit(X.to_dict('records'), y) 

/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[10:49:34] WARNING: /usr/local/miniconda/conda-bld/xgboost_1572315027083/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('ohe_onestep',
                 DictVectorizer(dtype=<class 'numpy.float64'>, separator='=',
                                sort=True, sparse=False)),
                ('xgb_model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0,
                              importance_type='gain', learning_rate=0.1,
                              max_delta_step=0, max_depth=3, min_child_weight=1,
                              missing=None, n_estimators=100, n_jobs=1,
                              nthread=None, objective='reg:linear',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, seed=None, silent=None,
                              subsample=1, verbosity=1))],
         verbose=False)

### Cross-validating your XGBoost model
In this exercise, you'll go one step further by using the pipeline you've created to preprocess and cross-validate your model.

In [37]:
df = ames.copy()
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [38]:
# Fill LotFrontage missing values with 0
X.LotFrontage = X.LotFrontage.fillna(0)

In [39]:
# Setup the pipeline steps: steps
steps = [("ohe_onestep", DictVectorizer(sparse=False)),
         ("xgb_model", xgb.XGBRegressor(max_depth=2, objective="reg:linear"))]

In [40]:
# Create the pipeline: xgb_pipeline
xgb_pipeline = Pipeline(steps)

In [41]:
# Cross-validate the model
cross_val_scores = cross_val_score(xgb_pipeline, X.to_dict('records'), y, scoring='neg_mean_squared_error', cv=10)

[10:51:19] WARNING: /usr/local/miniconda/conda-bld/xgboost_1572315027083/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:51:19] WARNING: /usr/local/miniconda/conda-bld/xgboost_1572315027083/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:51:20] WARNING: /usr/local/miniconda/conda-bld/xgboost_1572315027083/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[10:51:22] WARNING: /usr/local/miniconda/conda-bld/xgboost_1572315027083/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:51:23] WARNING: /usr/local/miniconda/conda-bld/xgboost_1572315027083/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:51:23] WARNING: /usr/local/miniconda/conda-bld/xgboost_1572315027083/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [44]:
# Print the 10-fold RMSE
print("10-fold RMSE: ", np.mean(np.sqrt(np.abs(cross_val_scores))))

10-fold RMSE:  29867.603720688923


### Kidney disease case study I: Categorical Imputer
You'll now continue your exploration of using pipelines with a dataset that requires significantly more wrangling. The chronic kidney disease dataset contains both categorical and numeric features, but contains lots of missing values. The goal here is to predict who has chronic kidney disease given various blood indicators as features.

As Sergey mentioned in the video, you'll be introduced to a new library, sklearn_pandas, that allows you to chain many more processing steps inside of a pipeline than are currently supported in scikit-learn. Specifically, you'll be able to impute missing categorical values directly using the Categorical_Imputer() class in sklearn_pandas, and the DataFrameMapper() class to apply any arbitrary sklearn-compatible transformer on DataFrame columns, where the resulting output can be either a NumPy array or DataFrame.

We've also created a transformer called a Dictifier that encapsulates converting a DataFrame using .to_dict("records") without you having to do it explicitly (and so that it works in a pipeline). Finally, we've also provided the list of feature names in kidney_feature_names, the target name in kidney_target_name, the features in X, and the target in y.

In this exercise, your task is to apply the CategoricalImputer to impute all of the categorical columns in the dataset. You can refer to how the numeric imputation mapper was created as a template. Notice the keyword arguments input_df=True and df_out=True? This is so that you can work with DataFrames instead of arrays. By default, the transformers are passed a numpy array of the selected columns as input, and as a result, the output of the DataFrame mapper is also an array. Scikit-learn transformers have historically been designed to work with numpy arrays, not pandas DataFrames, even though their basic indexing interfaces are similar.

In [383]:
X, y = kidney_disease.iloc[:, :-1], kidney_disease.iloc[:, -1]

In [384]:
# Check number of nulls in each feature column
nulls_per_column = X.isnull().sum()
print(nulls_per_column)

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wc       106
rc       131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
dtype: int64


In [385]:
# Create a boolean mask for categorical columns
categorical_feature_mask = X.dtypes == object

In [386]:
# Get list of categorical column names
categorical_columns = X.columns[categorical_feature_mask].tolist()

In [387]:
# Get list of non-categorical column names
non_categorical_columns = X.columns[~categorical_feature_mask].tolist()

In [388]:
# Apply numeric imputer
numeric_imputation = [
        ([numeric_feature], SimpleImputer(strategy="median")) for numeric_feature in non_categorical_columns]

numeric_imputation_mapper = DataFrameMapper(numeric_imputation, input_df=True, df_out=True)

In [389]:
# Apply categorical imputer
categorical_imputation = [(category_feature, CategoricalImputer()) for category_feature in categorical_columns]

categorical_imputation_mapper = DataFrameMapper(categorical_imputation, input_df=True, df_out=True)

### Kidney disease case study II: Feature Union
Having separately imputed numeric as well as categorical columns, your task is now to use scikit-learn's FeatureUnion to concatenate their results, which are contained in two separate transformer objects - numeric_imputation_mapper, and categorical_imputation_mapper, respectively.

You may have already encountered FeatureUnion in Machine Learning with the Experts: School Budgets. Just like with pipelines, you have to pass it a list of (string, transformer) tuples, where the first half of each tuple is the name of the transformer.

In [390]:
# Combine the numeric and categorical transformations (FeatureUnion not return DataFrame)
numeric_categorical_union = FeatureUnion([
                                          ("num_mapper", numeric_imputation_mapper),
                                          ("cat_mapper", categorical_imputation_mapper)
                                         ])

In [391]:
# Combine the numeric and categorical transformations (alternative way return DataFrame)
# https://stackoverflow.com/questions/52055658/sklearn-pandas-in-a-pipeline-returns-typeerror-builtin-function-or-method-obj
transformers = []
transformers.extend(numeric_imputation)
transformers.extend(categorical_imputation)

numeric_categorical_union_mapper = DataFrameMapper(transformers, input_df=True, df_out=True)

### Kidney disease case study III: Full pipeline
It's time to piece together all of the transforms along with an XGBClassifier to build the full pipeline!

Besides the numeric_categorical_union that you created in the previous exercise, there are two other transforms needed: the Dictifier() transform which we created for you, and the DictVectorizer().

After creating the pipeline, your task is to cross-validate it to see how well it performs.

In [392]:
# Custom transformer to convert Pandas DataFrame into Dict
class Dictifier(BaseEstimator, TransformerMixin):       
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.to_dict('records')

In [393]:
# Create full pipeline
pipeline = Pipeline([
#                      ("featureunion", numeric_categorical_union),
                     ("featureunion", numeric_categorical_union_mapper),
                     ("dictifier", Dictifier()),
                     ("vectorizer", DictVectorizer(sort=False)),
                     ("clf", xgb.XGBClassifier(max_depth=3))
                    ])

In [394]:
# Perform cross-validation
cross_val_scores = cross_val_score(pipeline, X, y, scoring="roc_auc", cv=3)

/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/sklearn/utils/validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"
/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/sklearn/utils/validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"
/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/sklearn/utils/validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"
/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/sklearn/utils/validation.py:931

In [395]:
# Print avg. AUC
print("3-fold AUC: ", np.mean(cross_val_scores))

3-fold AUC:  0.998637406769937


### Alternative Pipeline
https://medium.com/bigdatarepublic/integrating-pandas-and-scikit-learn-with-pipelines-f70eb6183696

In [396]:
# Transformer that does the column selection
class TypeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, dtype):
        self.dtype = dtype
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X.select_dtypes(include=[self.dtype])

In [397]:
# Transformer that apply CategoricalImputer to DataFrame
class DataFrameCategoricalImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X.apply(lambda x: CategoricalImputer().fit_transform(x))

In [398]:
# Pipeline for boolean type
boolean_pipeline = Pipeline([
        # Selecting booleans
        ('selector', TypeSelector('bool'))
])

In [399]:
# Pipeline for numerical type
numerical_pipeline = Pipeline([
        # Selecting numericals
        ('selector', TypeSelector(np.number)),
        
        # Imputing missing values
        ('imputer', SimpleImputer(strategy="median")),
        
        # Scaling
        ('scaler', StandardScaler())
])

In [400]:
# Pipeline for categorical type
categorical_pipeline = Pipeline([
        # Selecting categoricals
        ('selector', TypeSelector('category')),
        
        # Imputing missing values
        ('imputer', DataFrameCategoricalImputer()),
        
        
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [401]:
# Combine all types transformations
bool_numeric_categorical_union = FeatureUnion([
        ("boolean", boolean_pipeline),
        ("numerical", numerical_pipeline),
        ("categorical", categorical_pipeline)
])

In [402]:
# Create full pipeline
pipeline = Pipeline([
        ("features", bool_numeric_categorical_union),
        ("clf", xgb.XGBClassifier(max_depth=3))
])

In [403]:
# Perform cross-validation
cross_val_scores = cross_val_score(pipeline, X, y, scoring="roc_auc", cv=3)

In [404]:
# Print avg. AUC
print("3-fold AUC: ", np.mean(cross_val_scores))

3-fold AUC:  0.9987990055459934


### Bringing it all together
Alright, it's time to bring together everything you've learned so far! In this final exercise of the course, you will combine your work from the previous exercises into one end-to-end XGBoost pipeline to really cement your understanding of preprocessing and pipelines in XGBoost.

Your work from the previous 3 exercises, where you preprocessed the data and set up your pipeline, has been pre-loaded. Your job is to perform a randomized search and identify the best hyperparameters.

In [193]:
# Create the parameter grid
gbm_param_grid = {
    'clf__learning_rate': np.arange(0.05, 1, 0.05),
    'clf__max_depth': range(3, 10, 1),
    'clf__n_estimators': range(50, 200, 50)
}

In [196]:
# Perform RandomizedSearchCV
randomized_roc_auc = RandomizedSearchCV(estimator=pipeline, param_distributions=gbm_param_grid, 
                                        n_iter=2, scoring='roc_auc', cv=2, verbose=1)

In [197]:
# Fit the estimator
randomized_roc_auc.fit(X, y)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/sklearn/utils/validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"
/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/sklearn/utils/validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"
/Users/lukasz/anaconda/envs/data-camp/lib/python3.8/site-packages/sklearn/utils/validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"
/Users/lukasz/an

RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('featureunion',
                                              DataFrameMapper(default=False,
                                                              df_out=True,
                                                              features=[(['age'],
                                                                         SimpleImputer(add_indicator=False,
                                                                                       copy=True,
                                                                                       fill_value=None,
                                                                                       missing_values=nan,
                                                                                       strategy='median',
                                                                                       v

In [198]:
# Compute metrics
print(randomized_roc_auc.best_score_)
print(randomized_roc_auc.best_estimator_)

0.9978666666666667
Pipeline(memory=None,
         steps=[('featureunion',
                 DataFrameMapper(default=False, df_out=True,
                                 features=[(['age'],
                                            SimpleImputer(add_indicator=False,
                                                          copy=True,
                                                          fill_value=None,
                                                          missing_values=nan,
                                                          strategy='median',
                                                          verbose=0)),
                                           (['bp'],
                                            SimpleImputer(add_indicator=False,
                                                          copy=True,
                                                          fill_value=None,
                                                          missing_values=nan,
          